In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy

import mplsoccer
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.projections import get_projection_class
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [6]:
eventsPassDataLaLiga2019 = pd.read_csv("data/eventsPassdf.csv",
                                      low_memory=False)

In [4]:
passRelDataLaLiga2019 = pd.read_csv("data/eventsPassdf.csv",
                                    low_memory=False)

In [12]:
eventsPassDataLaLiga2019.groupby('player.id').agg({"player.name": "first",
                                                    "team.name": "first",
                                                    "type.id": "count",
                                                    "pass.through_ball": "sum",
                                                    "pass.shot_assist": "sum",
                                                    "pass.goal_assist": "sum"})\
.sort_values("pass.through_ball",ascending=False)

,player.name,team.name,type.id,pass.through_ball,pass.shot_assist,pass.goal_assist
player.id,,,,,,
5503.0,Lionel Andrés Messi Cuccittini,Barcelona,2142,96.0,67.0,19.0
5203.0,Sergio Busquets i Burgos,Barcelona,2280,25.0,16.0,1.0
8118.0,Frenkie de Jong,Barcelona,1363,16.0,17.0,1.0
5246.0,Luis Alberto Suárez Díaz,Barcelona,643,14.0,20.0,8.0
6379.0,Sergi Roberto Carnicer,Barcelona,1883,14.0,23.0,0.0
...,...,...,...,...,...,...
6698.0,Marko Dmitrović,Eibar,54,0.0,0.0,0.0
6693.0,Raúl Rodríguez Navas,Osasuna,15,0.0,1.0,0.0
6684.0,Roger Martí Salvador,Levante,24,0.0,2.0,0.0


In [18]:
playerWisedf = eventsPassDataLaLiga2019.groupby('player.id').agg({"player.name": "first",
                                                    "team.name": "first",
                                                    "type.id": "count",
                                                    "pass.through_ball": "sum",
                                                    "pass.shot_assist": "sum",
                                                    "pass.goal_assist": "sum"})

In [19]:
playerMatchMinsdf = eventPassDataLaLiga2019.drop_duplicates(subset=["player.id", "match_id"]).groupby(["player.id"])\
.agg({"minsPlayed": "sum"})

In [20]:
playerWisedf = pd.concat((playerWisedf,playerMatchMinsdf),axis=1)

In [21]:
playerWisedf

,player.name,team.name,type.id,pass.through_ball,pass.shot_assist,pass.goal_assist,minsPlayed
player.id,,,,,,,
2948.0,Nabil Fekir,Real Betis,17,0.0,0.0,0.0,75.0
3042.0,Mikel Merino Zazón,Real Sociedad,101,2.0,3.0,0.0,187.0
3058.0,Javier Hernández Balcázar,Sevilla,7,0.0,0.0,0.0,28.0
3122.0,Yannis Salibur,Mallorca,7,0.0,2.0,0.0,11.0
3139.0,Chidozie Collins Awaziem,Leganés,19,0.0,0.0,0.0,96.0
...,...,...,...,...,...,...,...
32953.0,Asier Villalibre Molina,Athletic Bilbao,2,0.0,0.0,0.0,26.0
34515.0,Fernando Niño Rodriguez,Villarreal,7,0.0,0.0,0.0,23.0
40881.0,Enrique Pérez Muñoz,Real Valladolid,41,0.0,1.0,0.0,92.0


In [22]:
playerWisedf.rename(columns={"type.id": "totalPasses",
                             "pass.through_ball": "passTB",
                             "pass.shot_assist": "passSA",
                             "pass.goal_assist": "passGA"},
                    inplace=True)

In [24]:
per90cols = ["totalPasses","passTB","passSA","passGA"]


In [25]:
playerWisedf.columns

Index(['player.name', 'team.name', 'totalPasses', 'passTB', 'passSA', 'passGA',
       'minsPlayed'],
      dtype='object')

In [26]:
for col in per90cols:
    playerWisedf[col+"per90"] = playerWisedf[col].divide(playerWisedf["minsPlayed"]).multiply(90)

In [27]:
playerWisedf.head()

,player.name,team.name,totalPasses,passTB,passSA,passGA,minsPlayed,totalPassesper90,passTBper90,passSAper90,passGAper90
player.id,,,,,,,,,,,
2948.0,Nabil Fekir,Real Betis,17,0.0,0.0,0.0,75.0,20.400000,0.000000,0.000000,0.0
3042.0,Mikel Merino Zazón,Real Sociedad,101,2.0,3.0,0.0,187.0,48.609626,0.962567,1.443850,0.0
3058.0,Javier Hernández Balcázar,Sevilla,7,0.0,0.0,0.0,28.0,22.500000,0.000000,0.000000,0.0
3122.0,Yannis Salibur,Mallorca,7,0.0,2.0,0.0,11.0,57.272727,0.000000,16.363636,0.0
3139.0,Chidozie Collins Awaziem,Leganés,19,0.0,0.0,0.0,96.0,17.812500,0.000000,0.000000,0.0


# Vision Rating

In [30]:
playerWisedf["visionRating"] = \
(playerWisedf["passTBper90"]*0.2)\
.add(playerWisedf["passSAper90"]*0.3)\
.add(playerWisedf["passGAper90"]*0.5)

In [31]:
playerWisedf["visionRating"].nlargest(10).index

Float64Index([ 3122.0, 24315.0, 30421.0, 23947.0,  5503.0, 23981.0, 11386.0,
               3308.0,  5552.0,  6885.0],
             dtype='float64', name='player.id')

In [32]:
playerWisedf.loc[playerWisedf["visionRating"].nlargest(10).index, ["player.name", "team.name", "visionRating"]]

,player.name,team.name,visionRating
player.id,,,
3122.0,Yannis Salibur,Mallorca,4.909091
24315.0,Enrique Barja Afonso,Osasuna,2.892857
30421.0,Matías Ezequiel Vargas Martín,Espanyol,1.968750
23947.0,Roberto Torres Morales,Osasuna,1.636364
5503.0,Lionel Andrés Messi Cuccittini,Barcelona,1.479784
23981.0,Álvaro Tejero Sacristán,Eibar,1.255814
11386.0,Santiago Cazorla González,Villarreal,1.167939
3308.0,Kieran Trippier,Atlético Madrid,1.161290
5552.0,Marcelo Vieira da Silva Júnior,Real Madrid,1.087912


In [34]:
playerWiseFiltereddf = playerWisedf[playerWisedf["minsPlayed"] > 45]

In [35]:
playerWiseFiltereddf["visionRating"] = \
(playerWiseFiltereddf["passTBper90"]*0.2)\
.add(playerWiseFiltereddf["passSAper90"]*0.3)\
.add(playerWiseFiltereddf["passGAper90"]*0.5)

<ipython-input-35-ebc331ddba4b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playerWiseFiltereddf["visionRating"] = \


In [37]:
playerWiseFiltereddf.loc[playerWiseFiltereddf["visionRating"].nlargest(10).index, ["player.name", "team.name", "visionRating"]]

,player.name,team.name,visionRating
player.id,,,
5503.0,Lionel Andrés Messi Cuccittini,Barcelona,1.479784
23981.0,Álvaro Tejero Sacristán,Eibar,1.255814
11386.0,Santiago Cazorla González,Villarreal,1.167939
3308.0,Kieran Trippier,Atlético Madrid,1.161290
5552.0,Marcelo Vieira da Silva Júnior,Real Madrid,1.087912
6392.0,Íñigo Lekue Martínez,Athletic Bilbao,1.058824
6685.0,Mikel Oyarzabal Ugarte,Real Sociedad,1.041322
6583.0,Carlos Soler Barragán,Valencia,0.910112
12073.0,Pablo Hervías Ruiz,Real Valladolid,0.910112


In [38]:
from sklearn.preprocessing import MinMaxScaler

In [42]:
scaler = MinMaxScaler((1, 10))
playerWiseFiltereddf["visionRating"] =\
    scaler.fit_transform(np.array(playerWiseFiltereddf["visionRating"]).reshape(-1, 1))

<ipython-input-42-4f9e643be498>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playerWiseFiltereddf["visionRating"] =\


In [43]:
playerWiseFiltereddf.loc[playerWiseFiltereddf["visionRating"].nlargest(10).index, ["player.name", "team.name", "visionRating"]]

,player.name,team.name,visionRating
player.id,,,
5503.0,Lionel Andrés Messi Cuccittini,Barcelona,10.000000
23981.0,Álvaro Tejero Sacristán,Eibar,8.637819
11386.0,Santiago Cazorla González,Villarreal,8.103366
3308.0,Kieran Trippier,Atlético Madrid,8.062930
5552.0,Marcelo Vieira da Silva Júnior,Real Madrid,7.616646
6392.0,Íñigo Lekue Martínez,Athletic Bilbao,7.439730
6685.0,Mikel Oyarzabal Ugarte,Real Sociedad,7.333288
6583.0,Carlos Soler Barragán,Valencia,6.535274
12073.0,Pablo Hervías Ruiz,Real Valladolid,6.535274


In [48]:
playerWisedf.to_csv("data/playerWise.csv")

In [47]:
playerWisedf.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 367 entries, 2948.0 to 41125.0
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   player.name       367 non-null    object 
 1   team.name         367 non-null    object 
 2   totalPasses       367 non-null    int64  
 3   passTB            367 non-null    float64
 4   passSA            367 non-null    float64
 5   passGA            367 non-null    float64
 6   minsPlayed        367 non-null    float64
 7   totalPassesper90  367 non-null    float64
 8   passTBper90       367 non-null    float64
 9   passSAper90       367 non-null    float64
 10  passGAper90       367 non-null    float64
 11  visionRating      367 non-null    float64
dtypes: float64(9), int64(1), object(2)
memory usage: 47.3+ KB
